In [1]:
import sqlite3

In [2]:
# Set up database
conn = sqlite3.connect('sp500_stocks.db') # Create or connect to the db file
cursor = conn.cursor() # Create a cursor to excute command

In [3]:
# Create neccessary tables
# Ticker info table
statement = """
    CREATE TABLE IF NOT EXISTS companies (
        ticker TEXT PRIMARY KEY,
        name TEXT NOT NULL,
        sector INTEGER,
        industry INTEGER,
        description TEXT,
        ipo_year INTEGER
    )
"""
cursor.execute(statement)


In [4]:
# Price info table
statement = """
    CREATE TABLE IF NOT EXISTS price (
    ticker TEXT,
    date DATE,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    adj_close REAL,
    volume INTEGER,
    PRIMARY KEY (ticker, date),
    FOREIGN KEY (ticker) REFERENCES companies(ticker)
    FOREIGN KEY (sector) REFERENCES sectors(sector_id),
    FOREIGN KEY (industry) REFERENCES industries(industry_id)
)
"""
cursor.execute(statement)

In [6]:
# Sector info table
statement = """
    CREATE TABLE IF NOT EXISTS sectors (
        sector_id INTEGER PRIMARY KEY AUTOINCREMENT,
        sector TEXT,
        description TEXT
    )
"""
cursor.execute(statement)

In [8]:
# Industry info table
statement = """
    CREATE TABLE IF NOT EXISTS industries (
        industry_id INTEGER PRIMARY KEY AUTOINCREMENT,
        sector INTEGER,
        industry TEXT,
        description TEXT,
        FOREIGN KEY (sector) REFERENCES sectors(sector_id)
    )"""
cursor.execute(statement)

In [ ]:
# Extract and print the schema of sp500_stocks.db
import sqlite3
conn = sqlite3.connect('sp500_stocks.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    print(f"Table: {table[0]}")
    cursor.execute(f"PRAGMA table_info({table[0]})")
    columns = cursor.fetchall()
    for col in columns:
        print(col)
    print()
conn.close()

Table: companies
(0, 'ticker', 'TEXT', 0, None, 1)
(1, 'name', 'TEXT', 1, None, 0)
(2, 'sector', 'TEXT', 0, None, 0)
(3, 'industry', 'TEXT', 0, None, 0)
(4, 'description', 'TEXT', 0, None, 0)
(5, 'ipo_year', 'INTEGER', 0, None, 0)

Table: price
(0, 'ticker', 'TEXT', 0, None, 1)
(1, 'date', 'DATETIME', 0, None, 2)
(2, 'open', 'REAL', 0, None, 0)
(3, 'high', 'REAL', 0, None, 0)
(4, 'low', 'REAL', 0, None, 0)
(5, 'close', 'REAL', 0, None, 0)
(6, 'adj_close', 'REAL', 0, None, 0)
(7, 'volume', 'INTEGER', 0, None, 0)

Table: sectors
(0, 'sector_id', 'INTEGER', 0, None, 1)
(1, 'sector', 'TEXT', 0, None, 0)
(2, 'description', 'TEXT', 0, None, 0)

Table: sqlite_sequence
(0, 'name', '', 0, None, 0)
(1, 'seq', '', 0, None, 0)

Table: industries
(0, 'industry_id', 'INTEGER', 0, None, 1)
(1, 'sector', 'INTEGER', 0, None, 0)
(2, 'industry', 'TEXT', 0, None, 0)
(3, 'description', 'TEXT', 0, None, 0)

